In [1]:
import os, ast
from os import listdir
from os.path import isfile, join

import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error

In [162]:
######################## AUDIO TRAINING #####################
#audio files include IEMOCAP data, Toronto Emotion Speech Dataset and IHMC Data

#read audio data features (csv file) from opensmile processing
with open('IEMOCAPvectorsfinal.txt') as t:
    audioIEMOCAP_features = [map(float,line.split()) for line in t]

for i in range(0,len(audioIEMOCAP_features)):
    audioIEMOCAP_features[i] = list(audioIEMOCAP_features[i])

#other audio: Toronto & IHMC
with open('otheraudiovectors.txt') as t:
    other_audio_features = [map(float,line.split()) for line in t]

for j in range(0,len(other_audio_features)):
    other_audio_features[j] = list(other_audio_features[j])

#read emotional labels from file
other_audio_labels = np.load('otheraudiolabelsfixed.npy')

#read filenames that has 
with open("IEMOCAP_filenames.txt") as f:
    fnames_with_feat = f.read().splitlines() 

#reading the labels from label file
line_dict = open("IEMOCAP_label_dict2.txt").read()

line_dict = ast.literal_eval(line_dict)
IEMOCAPlabels = []            
for i in fnames_with_feat:
    IEMOCAPlabels.append(line_dict[i])

In [163]:
#to remove all emotions that are not one of the following, find their indices and store them
takeouts = []
for i in range(0,len(IEMOCAPlabels)):
    if IEMOCAPlabels[i] != ['Happiness'] and IEMOCAPlabels[i] != ['Sadness'] and IEMOCAPlabels[i] != ['Fear'] and IEMOCAPlabels[i] != ['Anger']: 
        takeouts.append(i)

In [164]:
#remove unneeded labels/output info for the indices
tokeeplabels = []
for i in range(0,len(IEMOCAPlabels)):
    if i not in takeouts:
        tokeeplabels.append(IEMOCAPlabels[i])

In [166]:
#only keep samples/vectors of information with wanted emotions (happiness, joy, fear, sadness)
tokeepsamples = []
for i in range(0,len(audioIEMOCAP_features)):
    if i not in takeouts:
        tokeepsamples.append(audioIEMOCAP_features[i])

In [167]:
#some data preprocessing
for i in range(0,len(tokeeplabels)):
    if tokeeplabels[i] == ['Happiness']:
        tokeeplabels[i] = 'joy'
    if tokeeplabels[i] == ['Anger']:
        tokeeplabels[i] = 'anger'
    if tokeeplabels[i] == ['Fear']:
        tokeeplabels[i] = 'fear'
    if tokeeplabels[i] == ['Sadness']:
        tokeeplabels[i] = 'sadness'

In [169]:
X_train, X_test, y_train, y_test = train_test_split(tokeepsamples, tokeeplabels, test_size=0.2, random_state=42)
#X_test and y_test to be used for the actual test section later
#X_train and y_train to be added to other audio features and labels

In [170]:
#joining audio features from both samples (they're all of vector length ~1500)
audiofeatures = []
for i in range(0,len(X_train)):
    audiofeatures.append(X_train[i])

for j in range(0,len(other_audio_features)):
    audiofeatures.append(other_audio_features[j])

audiofeatures = np.asarray(audiofeatures)

#need to add 80% of IEMOCAP to other audio to get 
labels = np.append(y_train,other_audio_labels) #this append only works bc the labels are all single strings

In [172]:
#SVM classifier

#loading audio classifier
audioclf = svm.SVC(class_weight='balanced')
#AUDIO TRAINING: Toronto Data (100%), IHMC Berlin Data (100%), IEMOCAP Data (80%)
audioclf.fit(audiofeatures,labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [173]:
#we now use transcriptions of part of the audio we have for our text model
with open('IEMOCAP_transcriptions2.txt') as f:
    IEMOCAPtext = f.read().splitlines() 

In [174]:
#keep the transcriptions only of the emotions we want
tokeepsentences= []
for i in range(0,len(IEMOCAPtext)):
    if i not in takeouts:
        tokeepsentences.append(IEMOCAPtext[i])

In [175]:
#gets ontology vectors
with open('IEMOCAP_ont_vecs.txt') as t:
    IEMOCAP_ont_vecs = [map(float,line.split()) for line in t]
for i in range(0,len(IEMOCAP_ont_vecs)):
    IEMOCAP_ont_vecs[i] = list(IEMOCAP_ont_vecs[i])
IEMOCAP_ont_vecs = np.asarray(IEMOCAP_ont_vecs)

# gets w2v vectors
with open('avgIEMOCAPfinalw2v.txt') as t:
    IEMOCAP_w2v = [map(float,line.split()) for line in t]
for i in range(0,len(IEMOCAP_w2v)):
    IEMOCAP_w2v[i] = list(IEMOCAP_w2v[i])
IEMOCAP_w2v = np.asarray(IEMOCAP_w2v)

# averaging the two 
final_IEMOCAP_txt_vec = []
for v in range(len(IEMOCAP_w2v)):
    if np.count_nonzero(IEMOCAP_ont_vecs[v]) == 300:
        final_IEMOCAP_txt_vec.append(IEMOCAP_ont_vecs[v])
    else:
        temp_vec = (IEMOCAP_ont_vecs[v] + IEMOCAP_w2v[v]) / 2
        final_IEMOCAP_txt_vec.append(temp_vec)

In [177]:
#splitting IEMOCAP data specificially, because it must be used for test portion
X_trainItxt, X_testItxt, y_trainItxt, y_testItxt = train_test_split(final_IEMOCAP_txt_vec, tokeeplabels, test_size=0.2, random_state=42)

In [225]:
#also train NLP model with twitter emotion labeled data, SemEval Competition 2018
with open('twitter_vecs.txt') as t:
    twitter_vecs = [map(float,line.split()) for line in t]
for i in range(0,len(twitter_vecs)):
    twitter_vecs[i] = list(twitter_vecs[i])

In [227]:
#######TEXT TRAINING: Tweets (100%), IEMOCAP Data (80%) - PER EMOTION!!!!#############
textfeatures = []

#join together the two data types, features are vectors of length 300 w2v and ontology averages
for j in range(0,len(X_trainItxt)): #IEMOCAP 80%
    textfeatures.append(X_trainItxt[j])
    
for k in range(0,len(twitter_vecs)): #tweets
    textfeatures.append(twitter_vecs[k])

#open emotion labels and format
tweetlabels = []
with open('twitter_labels.txt') as inputfile:
    for row in csv.reader(inputfile):
        tweetlabels.append(row)
for i in range(0,len(tweetlabels)):
    tweetlabels[i] = tweetlabels[i][0]

In [230]:
#all tweet labels
addonlabels=[]
with open('alllabels.txt') as inputfile:
    for row in csv.reader(inputfile):
        addonlabels.append(row)

In [232]:
#preprocessing due to file glitch
for i in range(0,len(addonlabels)):
    if addonlabels[i] == ['a\tn\tg\te\tr']:
        tweetlabels.append('anger')
    elif addonlabels[i] == ['f\te\ta\tr']:
        tweetlabels.append('fear')
    elif addonlabels[i] == ['j\to\ty']:
        tweetlabels.append('joy')
    else:
        tweetlabels.append('sadness')
tweetlabels.append('sadness')

82199


In [233]:
#need to add 80% of IEMOCAP to other audio to get 
textlabels = np.append(y_trainItxt,tweetlabels) #this append only works bc the labels are all single strings

84647


In [234]:
#additional data we only have w2v for (no ontology)
w2v = np.loadtxt('w2vtweets.txt')

In [236]:
for i in range(0,len(w2v)):
    textfeatures.append(w2v[i])
textfeatures = np.asarray(textfeatures)

In [3]:
#loading text classifier
textclf = svm.SVC(class_weight='balanced')
textclf.fit(textfeatures,textlabels)
######################################################################################

In [ ]:
#format all data needed for regression portion
##################### ANGER ############################

with open('averageanger.txt') as t:
    XA_train = [map(float,line.split()) for line in t]

for i in range(0,len(XA_train)):
    XA_train[i] = list(XA_train[i])
XA_train = np.asarray(XA_train)

with open('angerscores.txt') as t:
    yatrain = [map(float,line.split()) for line in t]

for i in range(0,len(yatrain)):
    yatrain[i] = list(yatrain[i])
yatrain = np.asarray(yatrain)

####################   FEAR    ############################

with open('averagefear.txt') as t:
    XF_train = [map(float,line.split()) for line in t]

for i in range(0,len(XF_train)):
    XF_train[i] = list(XF_train[i])
XF_train = np.asarray(XF_train)

with open('fearscores.txt') as t:
    yftrain = [map(float,line.split()) for line in t]

for i in range(0,len(yftrain)):
    yftrain[i] = list(yftrain[i])
yftrain = np.asarray(yftrain)

####################   JOY    #############################

with open('averagejoy.txt') as t:
    XJ_train = [map(float,line.split()) for line in t]

for i in range(0,len(XJ_train)):
    XJ_train[i] = list(XJ_train[i])
XJ_train = np.asarray(XJ_train)

with open('joyscores.txt') as t:
    yjtrain = [map(float,line.split()) for line in t]

for i in range(0,len(yjtrain)):
    yjtrain[i] = list(yjtrain[i])
yjtrain = np.asarray(yjtrain)

#################### SADNESS ##############################

with open('averagesadness.txt') as t:
    XS_train = [map(float,line.split()) for line in t]

for i in range(0,len(XS_train)):
    XS_train[i] = list(XS_train[i])
XS_train = np.asarray(XS_train)

with open('sadnesscores.txt') as t:
    ystrain = [map(float,line.split()) for line in t]

for i in range(0,len(ystrain)):
    ystrain[i] = list(ystrain[i])
ystrain = np.asarray(ystrain)
###############################################################

In [240]:
#################    REGRESSION TRAINING  #####################
#anger split, model, and fitting
angerregression = svm.SVR(class_weight='balanced')
angerregression.fit(XA_train,yatrain.ravel()) 

#fear split, model, and fitting
fearregression = svm.SVR(class_weight='balanced')
fearregression.fit(XF_train,yftrain.ravel()) 

#joy split, model, and fitting
joyregression = svm.SVR(class_weight='balanced')
joyregression.fit(XJ_train,yjtrain.ravel()) 

#sadness model, and fitting
sadnessregression = svm.SVR(class_weight='balanced')
sadnessregression.fit(XS_train,ystrain.ravel())

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [241]:
######################### TEST PHASE ####################################
tpredicted = textclf.predict(X_testItxt)
apredicted = audioclf.predict(X_test)

In [243]:
agreed = 0
for i in range(0,len(tpredicted)):
    if tpredicted[i] == apredicted[i]: #only one regression model needed, they agreed!
        agreed +=1
        if tpredicted[i] == 'anger':
            intensity = angerregression.predict(X_testItxt[i].reshape(1, -1))
        elif tpredicted[i] == 'fear':
            intensity = fearregression.predict(X_testItxt[i].reshape(1, -1))
        elif tpredicted[i] == 'joy':
            intensity = joyregression.predict(X_testItxt[i].reshape(1, -1))
        else:
            intensity = sadnessregression.predict(X_testItxt[i].reshape(1, -1))
            
        print("This sample was found to be " + tpredicted[i] + " with intensity " + str(intensity))
    else: #they gave different emotions, need to do both
        if tpredicted[i] == 'anger':
            intensity1 = angerregression.predict(X_testItxt[i].reshape(1, -1))
        elif tpredicted[i] == 'fear':
            intensity1 = fearregression.predict(X_testItxt[i].reshape(1, -1))
        elif tpredicted[i] == 'joy':
            intensity1 = joyregression.predict(X_testItxt[i].reshape(1, -1))
        else:
            intensity1 = sadnessregression.predict(X_testItxt[i].reshape(1, -1))
            
        if apredicted[i] == 'anger':
            intensity2 = angerregression.predict(X_testItxt[i].reshape(1, -1))
        elif apredicted[i] == 'fear':
            intensity2 = fearregression.predict(X_testItxt[i].reshape(1, -1))
        elif apredicted[i] == 'joy':
            intensity2 = joyregression.predict(X_testItxt[i].reshape(1, -1))
        else:
            intensity2 = sadnessregression.predict(X_testItxt[i].reshape(1, -1))
            
        print("This sample was found have " + tpredicted[i] + " or " + apredicted[i] + " with intensity " + str(intensity1) + ", " + str(intensity2) + " respectively.")

This sample was found have anger or sadness with intensity [0.50210564], [0.50447284] respectively.
This sample was found have anger or sadness with intensity [0.50543008], [0.50645316] respectively.
This sample was found have anger or sadness with intensity [0.50859048], [0.50803195] respectively.
This sample was found have anger or sadness with intensity [0.50180541], [0.50374607] respectively.
This sample was found have anger or sadness with intensity [0.50772016], [0.50484636] respectively.
This sample was found have anger or joy with intensity [0.49707722], [0.4998168] respectively.
This sample was found have anger or sadness with intensity [0.49943695], [0.50444964] respectively.
This sample was found have anger or sadness with intensity [0.50464942], [0.50627339] respectively.
This sample was found have anger or sadness with intensity [0.49703734], [0.50354356] respectively.
This sample was found have anger or sadness with intensity [0.50533704], [0.50617511] respectively.
This 

This sample was found have anger or sadness with intensity [0.496833], [0.50351682] respectively.
This sample was found have anger or sadness with intensity [0.5028897], [0.50374931] respectively.
This sample was found have anger or sadness with intensity [0.50368504], [0.50503441] respectively.
This sample was found have anger or sadness with intensity [0.50742529], [0.50593143] respectively.
This sample was found have anger or sadness with intensity [0.49590642], [0.50374961] respectively.
This sample was found have anger or sadness with intensity [0.50394223], [0.50488512] respectively.
This sample was found have anger or sadness with intensity [0.50728077], [0.50743879] respectively.
This sample was found have anger or sadness with intensity [0.49593068], [0.50376196] respectively.
This sample was found have anger or sadness with intensity [0.50338285], [0.50543915] respectively.
This sample was found have anger or sadness with intensity [0.49656879], [0.50366988] respectively.
Thi

This sample was found have anger or sadness with intensity [0.5059612], [0.50728373] respectively.
This sample was found have anger or sadness with intensity [0.49604356], [0.50369554] respectively.
This sample was found have anger or sadness with intensity [0.49584271], [0.50375863] respectively.
This sample was found have anger or sadness with intensity [0.50341532], [0.50445342] respectively.
This sample was found have anger or sadness with intensity [0.49601247], [0.50372293] respectively.
This sample was found have anger or sadness with intensity [0.50609163], [0.50615637] respectively.
This sample was found have anger or sadness with intensity [0.4962206], [0.50369882] respectively.
This sample was found have anger or sadness with intensity [0.50416942], [0.50441428] respectively.
This sample was found have anger or sadness with intensity [0.49660558], [0.50379034] respectively.
This sample was found have anger or sadness with intensity [0.502204], [0.50415917] respectively.
This

This sample was found have anger or sadness with intensity [0.49583536], [0.50375059] respectively.
This sample was found have anger or sadness with intensity [0.50587709], [0.50682225] respectively.
This sample was found have anger or sadness with intensity [0.49609264], [0.50371998] respectively.
This sample was found have anger or sadness with intensity [0.49648387], [0.5037546] respectively.
This sample was found have anger or sadness with intensity [0.49607914], [0.50379201] respectively.
This sample was found to be anger with intensity [0.50269473]
This sample was found have anger or sadness with intensity [0.50378561], [0.50453164] respectively.
This sample was found have anger or sadness with intensity [0.50332728], [0.50501467] respectively.
This sample was found have anger or sadness with intensity [0.50500715], [0.50610717] respectively.
This sample was found have anger or sadness with intensity [0.4972068], [0.5035019] respectively.
This sample was found have anger or sadne

This sample was found have anger or sadness with intensity [0.49580367], [0.50377874] respectively.
This sample was found have anger or sadness with intensity [0.4959394], [0.50378767] respectively.
This sample was found have anger or sadness with intensity [0.50517861], [0.50585366] respectively.
This sample was found have anger or sadness with intensity [0.50657382], [0.50647987] respectively.
This sample was found have anger or sadness with intensity [0.50266973], [0.50386942] respectively.
This sample was found have anger or sadness with intensity [0.50744745], [0.50691943] respectively.
This sample was found have anger or sadness with intensity [0.49581055], [0.5037759] respectively.
This sample was found have anger or sadness with intensity [0.50393707], [0.50517242] respectively.
This sample was found have anger or sadness with intensity [0.50988604], [0.50802774] respectively.
This sample was found have anger or sadness with intensity [0.49589642], [0.50376803] respectively.
Th

This sample was found have anger or sadness with intensity [0.50491458], [0.50522343] respectively.
This sample was found to be anger with intensity [0.50571265]
This sample was found have anger or joy with intensity [0.5005384], [0.50011145] respectively.
This sample was found have anger or sadness with intensity [0.49602885], [0.50374196] respectively.
This sample was found have anger or sadness with intensity [0.4965004], [0.50367519] respectively.
This sample was found have anger or sadness with intensity [0.50399888], [0.50370476] respectively.
This sample was found have anger or sadness with intensity [0.50346801], [0.50397174] respectively.
This sample was found have anger or sadness with intensity [0.50493817], [0.50563018] respectively.
This sample was found have anger or sadness with intensity [0.5025084], [0.50381407] respectively.
This sample was found have anger or sadness with intensity [0.49696521], [0.50351715] respectively.
This sample was found have anger or sadness w

This sample was found have anger or sadness with intensity [0.50281377], [0.50297516] respectively.


In [247]:
agreed/len(tpredicted)

0.03431372549019608

In [248]:
from sklearn.metrics import accuracy_score
print(accuracy_score(tpredicted,y_test))
print(accuracy_score(apredicted,y_test))

0.3888888888888889
0.4068627450980392


In [249]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, tpredicted))
print(confusion_matrix(y_test, apredicted))

[[238   0   0   0]
 [ 17   0   0   0]
 [151   0   0   0]
 [206   0   0   0]]
[[ 21   0   0 217]
 [  0   0   0  17]
 [  0   0  22 129]
 [  0   0   0 206]]
